## prep Intern Data

In [ ]:
import pandas as pd 

In [ ]:
#read data
df_intern = pd.read_csv('/Users/faissalfarid/Desktop/bachelorarbeit/datasets/mainData/data_0.0_raw.csv', sep=';')

In [ ]:
#delete not relevant features
df_intern = df_intern.drop(['id', 'ocpptransactionid','created','authenticationtype',
                           'validationresult','isbillable','rfid','sigbeginmetervalue',
                           'sigendmetervalue','emppartner','cpopartner','platformname',
                           'clearingpartner','tariffuid','tariffname','chargepointmodel',
                           'chargepointvendor','ocppversion','firmwareversion','iccid',
                           'sim_iccid','imsi','sim_imsi','evseid','evseid','evseid',
                           'evseid','evseid','evseid','evseid','dsozaehlernummer',
                           'meterpublickey','chargingprotocol','voltage','current'
                           'chargingprotocol','maxamp', 'connectortype','Unnamed: 0',
                           'chargingstop','meterstart','meterstop',
                           'connectorid','chargingtype','phase','power',
                           'longitude','latitude','country',
                           'postalcode','city','region','street','housenumber',
                           ], axis = 1)


In [ ]:
#safe data
df_intern.to_csv('data_0.1_delNotRelevantFeatures.csv')
df_intern = pd.read_csv('/Users/faissalfarid/Desktop/bachelorarbeit/datasets/mainData/data_0.1_delNotRelevantFeatures.csv')

In [ ]:
#transformation 
#del all None values
#df_intern = df_intern.dropna()

#create new feature chargingtime
## transform to unixtime
df_intern['chargingstart'] = pd.to_datetime(df_intern['chargingstart'])
df_intern['chargingstop'] = pd.to_datetime(df_intern['chargingstop'])
df_intern['chargingtime'] = df_intern['chargingstop'] - df_intern['chargingstart']
df_intern['chargingtime'] = pd.to_timedelta(df_intern['chargingtime'])
df_intern['chargingtime'] = df_intern['chargingtime'].dt.total_seconds().div(60).astype(int)
#filter only german data
df_intern = df_intern[(df_intern['country'] == 'DE')]
#Rename collum names: 
df_intern.rename(columns={'chargingstart': 'date'}, inplace=True)

In [ ]:
#datareduction
item_counts = df_intern["chargingstation"].value_counts(normalize=True)
item_counts
df_intern = df_intern[(df_intern['chargingstation'] =='G120')]

## prep extern data

In [ ]:
#read extern data
preception = pd.read_csv('/Users/faissalfarid/Desktop/bachelorarbeit/datasets/wetherdata/fühlsbüttel/percipition/filteredpreception.csv')
wether = pd.read_csv('/Users/faissalfarid/Desktop/bachelorarbeit/datasets/wetherdata/fühlsbüttel/wether/filteredWetherData.csv')
sunshine = pd.read_csv('/Users/faissalfarid/Desktop/bachelorarbeit/datasets/wetherdata/fühlsbüttel/sunshine/filteredSunshineData.csv')

#change MESS_DATUM in date format
FORMAT='%Y%m%d%H'
#preception['MESS_DATUM'] = pd.to_datetime(preception['MESS_DATUM'], format=FORMAT)
#wether['MESS_DATUM'] = pd.to_datetime(wether['MESS_DATUM'], format=FORMAT)
sunshine['MESS_DATUM'] = pd.to_datetime(sunshine['MESS_DATUM'], format=FORMAT)


In [ ]:
#merge preception, wether, sunshine by date 
wether_preception = pd.merge(wether, preception, on="MESS_DATUM",how="left")
wether_preception['MESS_DATUM'] = pd.to_datetime(wether_preception['MESS_DATUM'])
df_extern = pd.merge(wether_preception, sunshine, on="MESS_DATUM",how="left")

In [ ]:
#drop irrelevant collums
df_extern = df_extern.drop(['Unnamed: 0_x', 'STATIONS_ID_x','QN_9','eor_x',
                           'Unnamed: 0_y','STATIONS_ID_y','QN_8','eor_y',
                           'Unnamed: 0','STATIONS_ID','QN_7','eor',
                           ], axis = 1)

## Find epsilon for DBSCAN

In [ ]:
#from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
rcParams['figure.figsize'] = 20,4
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('test.csv')

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = pd.to_numeric(df['date'])
X = df[['chargingtime', 'date']]
X = df[['date', 'chargingtime']].to_numpy()

neigh = NearestNeighbors(n_neighbors=4)
nbrs = neigh.fit(X)
distances,indices = nbrs.kneighbors(X)

distances = np.sort(distances, axis=0)

distances = distances[:,1]
plt.plot(distances)

plt.show()
print("The optimal value for epsilon will be found at the point of maximum curvature.")

In [ ]:
arr_zeros = np.zeros((len(distances),2))
arr_zeros = np.zeros((len(distances),2))

for i in range(len(arr_zeros)):
    arr_zeros[i][0] = i
    arr_zeros[i][1] = distances[i]

In [ ]:
X = df[['date', 'chargingtime']].to_numpy()
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X)
distances,indices = nbrs.kneighbors(X)
distances = np.sort(distances, axis=0)
distances

In [ ]:
from kneebow.rotor import Rotor

rotor = Rotor()
rotor.fit_rotate(arr_zeros)
elbow_idx = rotor.get_elbow_index()
print(elbow_idx)  # 11
rotor.plot_elbow()
plt.xlabel("index")
plt.ylabel("neighbour distance")
#safe plt image
plt.savefig('test.pdf', dpi=600,bbox_inches = 'tight')
plt.show()

## DSBCA Filter Anomaly

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
df = pd.read_csv('test.csv')
df['Anomaly'] = 0
df['date'] = pd.to_datetime(df['date'])

data = df
data_time = data
import numpy as np 
from numpy import random, where
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
#rcParams['figure.figsize'] = 8,6
rcParams['figure.figsize'] = 14,4

INIT = 1
anom_index = INIT

# Do the process so long until no anomaly were detected
while(np.count_nonzero(anom_index)>0):
    #prepare data for model 
    dbscan_data = data[['date', 'chargingtime']]
    df = dbscan_data

    #cause dbscan acept only floats make chargingstart numeric
    dbscan_data['date']= pd.to_numeric(dbscan_data['date'])

    #dbscan_data = dbscan_data.values.astype('float32', copy=False)
    dbscan_data
    #normalize Data 
    dbscan_data_scaler = StandardScaler().fit(dbscan_data)
    dbscan_data = dbscan_data_scaler.transform(dbscan_data)
    X = dbscan_data

    plt.scatter(X[:,0], X[:,1])
    plt.show()

    clustering = DBSCAN(eps=0.4, min_samples=4).fit(X)
    print(clustering.labels_)
    dbscan = DBSCAN(eps = 0.4, min_samples = 4)
    print(dbscan)

    pred = dbscan.fit_predict(X)
    anom_index = where(pred == -1)
    print("anomalys = ", np.count_nonzero(anom_index))
    values = X[anom_index]

    plt.scatter(X[:,0], X[:,1])
    plt.scatter(values[:,0], values[:,1], color='r')
    plt.xlabel("normalize date")
    plt.ylabel("normalize chargingtime")
    plt.show()

#################Mark all Anomalys##################
    ANOMALY = 888
    lenArr = np.count_nonzero(anom_index)    
    for i in range(lenArr):
        data.iloc[anom_index[0][i],data.columns.get_loc('Anomaly')] = ANOMALY
        data_time.iloc[anom_index[0][i],data_time.columns.get_loc('Anomaly')] = ANOMALY
        
#################Filter Anomalys##################        
    data = data[(data['Anomaly'] !=ANOMALY)]    
    data_time = data_time[(data_time['Anomaly'] !=ANOMALY)]  
    
    

## Scaling

In [ ]:
import pandas as pd
#frequenz ändern 
##load Data and set index to chargingstart 
df_intern = pd.read_csv('afterDbscanTest.csv',parse_dates=[2], index_col=[2])

#resample and sum chargingtime
df_intern = df_intern.resample('H').agg(dict(chargingtime='sum'))

#safe resampled dataset

for i in range(len(df_intern)-1):
    chargingtime = df_intern.iloc[i,df_intern.columns.get_loc('chargingtime')]
    if(chargingtime>120):
        df_intern.iloc[i,df_intern.columns.get_loc('chargingtime')] = 120
        df_intern.iloc[i+1,df_intern.columns.get_loc('chargingtime')] = chargingtime-120
df_intern = df_intern.reset_index(drop=False)        
df_intern.to_csv('test_after_freq.csv') 

## create new attr

In [ ]:
import pandas as pd 
df = pd.read_csv('test_after_freq.csv', parse_dates=[1])
df['hour'] = df['date'].dt.hour
df['weekday'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

#calc weekend
df['weekend'] = 88
for i in range(len(df)):
    day = df.iloc[i,df.columns.get_loc('weekday')]
    
    if(day >= 5 ):
        df.iloc[i,df.columns.get_loc('weekend')] = 1
    else:
        df.iloc[i,df.columns.get_loc('weekend')] = 0

#create occupation
LADEPUNKTE = 2 #EVSES
df['occupation'] = (df['chargingtime'] / (LADEPUNKTE*60))

#calcutae categorie occupation
df['categorieOcc'] = 1

for i in range(len(df)):
    occ = df.iloc[i,df.columns.get_loc('occupation')]
    
    if(occ >= 0.75):
        df.iloc[i,df.columns.get_loc('categorieOcc')] = 1
    elif(occ >= 0.5 and occ < 0.75):
        df.iloc[i,df.columns.get_loc('categorieOcc')] = 2 
    elif(occ >= 0.25 and occ < 0.5): 
        df.iloc[i,df.columns.get_loc('categorieOcc')] = 3
    else:
        df.iloc[i,df.columns.get_loc('categorieOcc')] = 4

#calcuate binary occupation
#df['binaryOcc'] = df['occupation'] < 0.5


#bei corr problemem anwenden
df['occupation'] = df['occupation']*100
df['occupation'] = df['occupation'].astype(int)



In [ ]:
df.to_csv('final_intern_data.csv')

## merge intern and extern Data 

In [ ]:
##########HHHHHHHOOOOOOOUUUUURRRRLLLLLLYYYYYYY##########

#load extern data
extern_data = pd.read_csv('final_extern_data.csv')
#load intern Data
intern_data = pd.read_csv('final_intern_data.csv')

#rename collum for extern_data same like intern_data for merging
intern_data['date'] = pd.to_datetime(intern_data['date'])
extern_data['date'] = pd.to_datetime(extern_data['date'])

intern_data['date'] = pd.to_datetime(intern_data['date'], utc = True)
extern_data['date'] = pd.to_datetime(extern_data['date'], utc = True)

data = pd.merge(intern_data, extern_data, on="date",how="left")
del(data['Unnamed: 0_x'])
del(data['Unnamed: 0_y'])
data.to_csv('final_data.csv')

In [ ]:
wetherData.to_csv('final_extern_data.csv')

In [ ]:
#rename collum for extern_data same like intern_data for merging
extern_data.rename(columns={'MESS_DATUM': 'date'}, inplace=True)
intern_data['date'] = pd.to_datetime(intern_data['date'])
extern_data['date'] = pd.to_datetime(extern_data['date'])

intern_data['date'] = pd.to_datetime(intern_data['date'], utc = True)
extern_data['date'] = pd.to_datetime(extern_data['date'], utc = True)

In [ ]:
#prep extern_data
del(extern_data['Unnamed: 0'])
del(extern_data['RF_TU'])
del(extern_data['WRTR'])

extern_data.rename(columns={'TT_TU': 'temperature'}, inplace=True)
extern_data.rename(columns={'R1': 'precipitation'}, inplace=True)
extern_data.rename(columns={'RS_IND': 'bool_precipitation'}, inplace=True)
extern_data.rename(columns={'SD_SO': 'sunshine'}, inplace=True)

import numpy as np
extern_data['  R1'] = extern_data['  R1'].replace(np.nan, 0.0)
extern_data['RS_IND'] = extern_data['RS_IND'].replace(np.nan, 0.0)
extern_data['SD_SO'] = extern_data['SD_SO'].replace(np.nan, 0.0)

extern_data.to_csv('final_extern_data.csv')

## PCA 

In [10]:
import pandas as pd
data = pd.read_csv('final_data.csv')
data = data.dropna()
data_pca = data.drop(['Unnamed: 0', 'date','occupation','categorieOcc',
                           ], axis = 1)


#Principle Component Analysis PCA
from sklearn.decomposition import PCA 
X = data_pca

pca = PCA(n_components=1) 
pca.fit(X)
#pca.components_
compressed_data = pca.transform(X)
compressed_data

array([[-7.73001805],
       [-7.72956189],
       [-7.54552154],
       ...,
       [-7.16342389],
       [-7.15989634],
       [-7.16005446]])

In [12]:
data['compressed_data'] = compressed_data

In [15]:
data_pca['compressed_data'] = compressed_data

In [17]:
data_pca

,hour,weekday,month,year,weekend,temperature,precipitaion,bool_precipitation,sunshine,compressed_data
0,12,0,10,2018,0,14.8,0.0,0.0,60.0,-7.730018
1,13,0,10,2018,0,14.8,0.0,0.0,60.0,-7.729562
2,14,0,10,2018,0,15.1,0.0,0.0,36.0,-7.545522
3,15,0,10,2018,0,14.7,0.0,0.0,49.0,-7.643048
4,16,0,10,2018,0,13.6,0.0,0.0,0.0,-7.257254
...,...,...,...,...,...,...,...,...,...,...
21895,19,2,4,2021,0,3.7,0.0,0.0,0.0,-7.171093
21896,20,2,4,2021,0,3.5,0.0,0.0,0.0,-7.169409
21897,21,2,4,2021,0,2.6,0.0,0.0,0.0,-7.163424
21898,22,2,4,2021,0,2.1,0.0,0.0,0.0,-7.159896


## Correlation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [8]:
df = pd.read_csv('final_data.csv')

In [9]:
#correlation
df.corr()

,Unnamed: 0,hour,weekday,month,year,weekend,occupation,categorieOcc,temperature,precipitaion,bool_precipitation,sunshine
Unnamed: 0,1.000000,-0.000345,0.000477,-0.072999,0.923525,0.001497,-0.038144,0.036849,-0.048331,-0.062567,-0.062649,0.002056
hour,-0.000345,1.000000,-0.000712,0.000691,-0.001552,-0.000075,0.179899,-0.174198,0.122706,-0.006842,-0.006867,0.029487
weekday,0.000477,-0.000712,1.000000,-0.004427,0.001938,0.790623,-0.013451,0.013964,0.002662,0.005427,0.005288,-0.000319
month,-0.072999,0.000691,-0.004427,1.000000,-0.448762,-0.000035,0.006296,-0.007198,0.209068,0.101844,0.101748,-0.046062
year,0.923525,-0.001552,0.001938,-0.448762,1.000000,0.001236,-0.036702,0.035883,-0.124341,-0.095859,-0.095889,0.018436
weekend,0.001497,-0.000075,0.790623,-0.000035,0.001236,1.000000,-0.029167,0.030219,0.005041,0.011549,0.011417,0.012336
occupation,-0.038144,0.179899,-0.013451,0.006296,-0.036702,-0.029167,1.000000,-0.989569,-0.013278,-0.000567,-0.000591,-0.036356
categorieOcc,0.036849,-0.174198,0.013964,-0.007198,0.035883,0.030219,-0.989569,1.000000,0.013022,0.002850,0.002881,0.034797
temperature,-0.048331,0.122706,0.002662,0.209068,-0.124341,0.005041,-0.013278,0.013022,1.000000,0.091566,0.090955,0.437448
precipitaion,-0.062567,-0.006842,0.005427,0.101844,-0.095859,0.011549,-0.000567,0.002850,0.091566,1.000000,0.999972,0.037384


In [ ]:
#autocorrelation
#source = https://www.youtube.com/watch?v=y8opUEd05Dg&ab_channel=ritvikmath
from statsmodels.graphics.tsaplots import plot_acf
pacf_plot = plot_pacf(df.occupation, lags = 1000)
#pd.plotting.autocorrelation_plot(df['occupation']).set_xlim([0, 1000])

In [ ]:
#extract trend 
from random import randrange
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
series = df['occupation']
result = seasonal_decompose(series.tail(1000), model='additive', period=50)
result.plot()
pyplot.show()

In [ ]:
import statsmodels.api as sm
decomposition = sm.tsa.seasonal_decompose(df['occupation'], model='additive')
fig = decomposition.plot()
from pylab import rcParams
#rcParams['figure.figsize'] = 8,6
rcParams['figure.figsize'] = [9.0, 5.0]